In [6]:
import os
import pandas as pd
import lightgbm as lgb
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback
import wandb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, root_mean_squared_error, r2_score
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit

In [2]:
load_dotenv()
os.environ["WANDB_API_KEY"] = os.getenv("WANDB_API_KEY")
os.environ["WANDB_DIR"] = "../data/wandb_logs"
wandb.login(key=os.environ["WANDB_API_KEY"])

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/brupesh/.netrc
wandb: Currently logged in as: brupeshmit (brupeshmit-massachusetts-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
train = pd.read_parquet("../data/train_clean_v3.parquet")
test = pd.read_parquet("../data/test_clean_v3.parquet")
target_col = "label"
y = train[target_col]
X = train.drop(columns=[target_col])

test_ids = test.iloc[:, 0]
test = test.drop(columns=[test.columns[0]])

feature_df = pd.read_csv("../data/resources/lgbm_feature_importance.csv")
top_features = feature_df.sort_values(by="importance", ascending=False)["feature"].iloc[:400].tolist()
X = X[top_features]
test = test[top_features]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_val, label=y_val)

In [7]:
tscv = TimeSeriesSplit(n_splits=5)

In [8]:
def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 16, 512),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "feature_pre_filter": False,
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "device": "gpu",
        "gpu_platform_id": 0,
        "gpu_device_id": 0,
        "verbosity": -1
    }

    rmse_scores = []
    train_rmse_scores = []
    r2_scores = []
    pearson_scores = []
    overfit_scores = []

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dval = lgb.Dataset(X_val, label=y_val)

        model = lgb.train(
            params,
            dtrain,
            num_boost_round=500,
            valid_sets=[dval],
            callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
        )

        preds_val = model.predict(X_val)
        preds_train = model.predict(X_train)

        rmse_val = root_mean_squared_error(y_val, preds_val)
        rmse_train = root_mean_squared_error(y_train, preds_train)
        r2_val = r2_score(y_val, preds_val)
        pearson = pd.Series(preds_val).corr(pd.Series(y_val))
        overfit = rmse_val - rmse_train

        rmse_scores.append(rmse_val)
        train_rmse_scores.append(rmse_train)
        r2_scores.append(r2_val)
        pearson_scores.append(pearson)
        overfit_scores.append(overfit)

    wandb.log({
        "avg_rmse": np.mean(rmse_scores),
        "avg_train_rmse": np.mean(train_rmse_scores),
        "avg_r2": np.mean(r2_scores),
        "avg_pearson": np.mean(pearson_scores),
        "avg_overfit": np.mean(overfit_scores)
    }, step=trial.number)

    return np.mean(rmse_scores)


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, callbacks=[WeightsAndBiasesCallback()])

[I 2025-08-14 23:01:05,701] A new study created in memory with name: no-name-e75d5dac-fe5c-4937-bd0c-dbfc1607d144
/tmp/ipykernel_36282/2956294688.py:69: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  study.optimize(objective, n_trials=30, callbacks=[WeightsAndBiasesCallback()])


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[16]	valid_0's rmse: 0.978829
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 1.00556
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 1.00975
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[6]	valid_0's rmse: 0.97596
Training until validation scores don't improve for 50 rounds


[I 2025-08-14 23:01:19,263] Trial 0 finished with value: 1.0067690801216946 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.007669544406706384, 'num_leaves': 334, 'max_depth': 10, 'min_child_samples': 72, 'subsample': 0.9551029921658695, 'colsample_bytree': 0.5362309999938235, 'reg_alpha': 0.0006823700742757026, 'reg_lambda': 0.0005581981230231926}. Best is trial 0 with value: 1.0067690801216946.


Early stopping, best iteration is:
[1]	valid_0's rmse: 1.06375


/home/brupesh/Documents/kaggle-drw-crypto/.venv/lib/python3.12/site-packages/lightgbm/callback.py:333: UserWarning: Early stopping is not available in dart mode
  _log_warning("Early stopping is not available in dart mode")


...

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.979435
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[14]	valid_0's rmse: 1.00415
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 1.00953
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[4]	valid_0's rmse: 0.976095
Training until validation scores don't improve for 50 rounds


[I 2025-08-14 23:16:17,699] Trial 28 finished with value: 1.0061479420361872 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.009049660698918067, 'num_leaves': 163, 'max_depth': 5, 'min_child_samples': 28, 'subsample': 0.6982100405540893, 'colsample_bytree': 0.9995693716450049, 'reg_alpha': 0.0001365869371658507, 'reg_lambda': 4.3643017456046687e-07}. Best is trial 22 with value: 1.0056800781801118.


[100]	valid_0's rmse: 1.07445
Early stopping, best iteration is:
[51]	valid_0's rmse: 1.06153
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.979317
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 1.00569
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 1.0095
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2]	valid_0's rmse: 0.976132
Training until validation scores don't improve for 50 rounds


[I 2025-08-14 23:16:28,229] Trial 29 finished with value: 1.0068639458931326 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.00578176009636947, 'num_leaves': 109, 'max_depth': 9, 'min_child_samples': 78, 'subsample': 0.6093566522343654, 'colsample_bytree': 0.9075907091169069, 'reg_alpha': 2.923655545640154e-05, 'reg_lambda': 0.00032012960707583995}. Best is trial 22 with value: 1.0056800781801118.


Early stopping, best iteration is:
[1]	valid_0's rmse: 1.06369


In [9]:
print(f"Best trial number: {study.best_trial.number}")
print(f"Best avg RMSE: {study.best_value}")
print("Best trial params:")
for k, v in study.best_trial.params.items():
    print(f"  {k}: {v}")

Best trial number: 22
Best avg RMSE: 1.0056800781801118
Best trial params:
  boosting_type: gbdt
  learning_rate: 0.04895956308561634
  num_leaves: 58
  max_depth: 3
  min_child_samples: 15
  subsample: 0.8953432004915698
  colsample_bytree: 0.9597250236055926
  reg_alpha: 1.9186865042305896e-05
  reg_lambda: 6.922591585909344e-05


In [10]:
best_params = study.best_trial.params
best_params.update({"objective": "regression", "metric": "rmse", "device": "gpu", "gpu_platform_id": 0, "gpu_device_id": 0})
model = lgb.train(
    best_params,
    dtrain,
    num_boost_round=1000,
    valid_sets=[dval],
    callbacks=[
        lgb.early_stopping(50),
        lgb.log_evaluation(100)
    ]
)

model.save_model("../data/models/lightgbm_opt_model_1.txt")

Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.93846
[200]	valid_0's rmse: 0.892449
[300]	valid_0's rmse: 0.857334
[400]	valid_0's rmse: 0.829413
[500]	valid_0's rmse: 0.80452
[600]	valid_0's rmse: 0.780467
[700]	valid_0's rmse: 0.759629
[800]	valid_0's rmse: 0.740964
[900]	valid_0's rmse: 0.723858
[1000]	valid_0's rmse: 0.707923
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 0.707923


In [11]:
preds = model.predict(test)
pd.DataFrame({"id": test_ids, "prediction": preds}).to_csv("../data/preds/lightgbm_opt1_preds.csv", index=False)

wandb.finish()

avg_overfit,▁▁█▄▁▁▁▅█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁
avg_r2,██▁▆███▅▃█████████████████▃███
avg_rmse,▁▁█▄▁▁▁▄▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁
avg_train_rmse,██▁▄███▄▁█████████████████▃███
colsample_bytree,▁▃▄▁▆▃▅▄▁▄▇▇█▆▇▆▇█▆▆▇▇▇▆▇▆▅██▇
learning_rate,▁▁▆▁▁▁▁▂▃▁▂▂▁▂▄▂▁▃█▁▂▃▃▃▄▁▃▂▁▁
max_depth,▆▃▆▇▃▆▃▇█▇▁▁▁▃▅▂▅▃▂▄▂▁▁▃▃▂▄▁▃▆
min_child_samples,▆▅▅█▄▅█▅▆▆▂▂▂▃▁▃▃▂▁▄▃▂▂▃▁▄▃▂▃▇
num_leaves,▆▅▂▆▃▆▆▅█▃▁▁▃▁▃▄▂▂▇▄▃▁▂▂▂▃▄▂▃▂
reg_alpha,▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁
reg_lambda,▁▁▁▁▁▁▁█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
